# Creating the parquet dataset from SQLite tables

In [2]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

!pwd

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/data


In [3]:
import torch
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.data.foundational_loader import FoundationalDataModule
import logging
import time

torch.manual_seed(1337)

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")


Using device: cuda.


In [4]:
# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../modelling/SurvStreamGPT/confs", job_name="dataset_creation_notebook"):
    cfg = compose(config_name="config_CompetingRisk129M", overrides=[])
print(OmegaConf.to_yaml(cfg))

is_decoder: true
data:
  batch_size: 64
  unk_freq_threshold: 0.0
  min_workers: 20
  global_diagnoses: false
  path_to_db: /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/
  path_to_ds: /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/
  meta_information_path: /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information.pickle
experiment:
  project_name: SurvStreamGPT_${head.SurvLayer}
  run_id: PreTrain_${head.SurvLayer}_129M_${experiment.seed}
  train: true
  test: true
  verbose: true
  seed: 1337
  log: true
  log_dir: /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/
  ckpt_dir: /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/
optim:
  num_epochs: 1
  learning_rate: 0.0001
  scheduler: CAWarmRestarts
  scheduler_periods: 5000
  scheduler_warmup: true
  lr_cosine_decay_period: 10000000.0
  val_check

In [9]:
# Build 
dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                            path_to_ds=cfg.data.path_to_ds,
                            load=False,
                            include_diagnoses=True,                            
                            include_measurements=True,
                            drop_missing_data=False,
                            drop_empty_dynamic=True,
                            tokenizer="tabular",
                            practice_inclusion_conditions=["COUNTRY = 'E'"],
                            overwrite_meta_information=cfg.data.meta_information_path,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size

print(f"{len(dm.train_set)} training patients")
print(f"{len(dm.val_set)} validation patients")
print(f"{len(dm.test_set)} test patients")
print(f"{vocab_size} vocab elements")

INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_test.pickle
INFO:root:Using val file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_val.pickle
INFO:root:Tokenzier created based on 3584.43M tokens
INFO:root:Using tabular tokenizer, created from meta information and containing 184 tokens
INFO:root:Loaded /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/split=train/ dataset, with 23,343,104 samples
INFO:root:Loaded /rds/projects/g/gokhal

23343104 training patients
1233601 validation patients
1263168 test patients
184 vocab elements


In [7]:
import polars as pl
pl.Config.set_tbl_rows(200)
pl.Config.set_fmt_str_lengths(100)
print(dm.train_set.tokenizer._event_counts)

shape: (173, 3)
┌──────────────────────────────────────────────────────────┬───────────┬───────────┐
│ EVENT                                                    ┆ COUNT     ┆ FREQUENCY │
│ ---                                                      ┆ ---       ┆ ---       │
│ str                                                      ┆ u32       ┆ f64       │
╞══════════════════════════════════════════════════════════╪═══════════╪═══════════╡
│ UNK                                                      ┆ 151553    ┆ 0.000042  │
│ HIVAIDS                                                  ┆ 41951     ┆ 0.000012  │
│ PSORIATICARTHRITIS2021                                   ┆ 51273     ┆ 0.000014  │
│ MS                                                       ┆ 53204     ┆ 0.000015  │
│ Plasma_N_terminal_pro_B_type_natriuretic_peptide_conc_70 ┆ 53534     ┆ 0.000015  │
│ LEUKAEMIA_PREVALENCEV2                                   ┆ 54438     ┆ 0.000015  │
│ N_terminal_pro_brain_natriuretic_peptide_level_

# Test data loading times (so we can optimise cpu usage)

In [28]:
import pyarrow.parquet as pq
import time

print(cfg.data.path_to_ds + "split=train/")
dataset1 = pq.ParquetDataset(cfg.data.path_to_ds + "split=train/", 
                            filters=[('PRACTICE_ID','=','20040')]
                            )

# Time to read 
start = time.time()   # starting time
df  = dataset1.read().to_pandas()
print(df[df["row_nr"] == df.row_nr[0]])
print(time.time() - start)


/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/split=train/



KeyboardInterrupt



## Time to load individual samples

In [24]:
from tqdm import tqdm
import numpy as np

times = []
start = time.time()   # starting time
for row_idx, row in enumerate(tqdm(dm.train_set)):
    # print(f"Sample loaded in {time.time()-start} seconds")
    times.append(time.time()-start)
    start = time.time()
    if row_idx > 100:
        break
print(np.mean(times))

  0%|          | 101/23343104 [00:04<299:32:25, 21.65it/s]

0.045749325378268375


## Time to load batch (with only one worker)

In [26]:
times = []
start = time.time()   # starting time
for batch_idx, batch in enumerate(tqdm(dm.train_dataloader())):
    # print(f"batch loaded in {time.time()-start} seconds")    
    times.append(time.time()-start)
    start = time.time()
    if batch_idx > 2:
        break
print(np.mean(times))

# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

  0%|          | 3/364736 [00:28<954:26:42,  9.42s/it]

5.817609012126923


In [27]:
dm.train_set.view_sample(1236, max_dynamic_events=12, report_time=True)

Time to retrieve sample index 1236 was 0.0607302188873291 seconds

SEX                 | F
IMD                 | 4.0
ETHNICITY           | ASIAN
birth_year          | 2013.0

Token                                                                      | Age               | Standardised value
O_E___height_1                                                             | 47                | nan               
O_E___weight_2                                                             | 47                | nan               
O_E___weight_2                                                             | 1468              | nan               
Body_mass_index_3                                                          | 1503              | -0.39             
O_E___height_1                                                             | 1503              | nan               
O_E___weight_2                                                             | 1503              | nan               
GFR_calculate